<a href="https://colab.research.google.com/github/aryajani/algo-trading/blob/main/algotrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import auth
from google.auth import default
import gspread



# Define the ticker symbol for Nifty 50
nifty_ticker = "^NSEI"

# Fetch data for Nifty 50
nifty_data = yf.Ticker(nifty_ticker)

# Get historical market data (CHANGE TO 10 YEARS)
historical_data = nifty_data.history(start="2014-01-01", end="2024-12-31")  # Fetch data for the past month

# Drop unwanted columns: 'Volume', 'Dividends', 'Stock Splits'
cleaned_data = historical_data.drop(columns=["Volume", "Dividends", "Stock Splits"])

cleaned_data.index = cleaned_data.index.astype(str)
cleaned_data = cleaned_data.reset_index()

# margin
margin = 100000

# 20SMA
cleaned_data["20SMA"] = cleaned_data["Close"].rolling(window=20).mean()

#100SMA
cleaned_data["100SMA"] = cleaned_data["Close"].rolling(window=100).mean()

# Create a Signal column to detect crossovers
def detect_signal(data):
    signals = []
    for i in range(len(data)):
        if i == 0:  # First row, no crossover to check
            signals.append("Hold")
        else:
            # Check for crossovers
            prev_20sma = data["20SMA"].iloc[i - 1]
            prev_100sma = data["100SMA"].iloc[i - 1]
            curr_20sma = data["20SMA"].iloc[i]
            curr_100sma = data["100SMA"].iloc[i]

            if prev_20sma <= prev_100sma and curr_20sma > curr_100sma:
                signals.append("Long")  # Bullish crossover
            elif prev_20sma >= prev_100sma and curr_20sma < curr_100sma:
                signals.append("Short")  # Bearish crossover
            else:
                signals.append("Hold")  # No crossover
    return signals


# Apply the function to detect signals
cleaned_data["Signal"] = detect_signal(cleaned_data)

# Profit
# Initialize a variable to track the entry price
entry_price = None

# Create a new column 'Profit Percentage' to store the profit percentage after each valid Sell
profP = []
profV = []

# Loop through the data to calculate profit percentages after each Sell signal
for i in range(len(cleaned_data)):

    curr_pos = cleaned_data["Signal"].iloc[i]
    exit_price = cleaned_data["Close"].iloc[i]
    if curr_pos == "Long" or curr_pos == "Short":
        prev_pos = curr_pos
        if not entry_price:
            entry_price = exit_price
            profP.append(None)
            profV.append(None)
        else:
            q = margin/entry_price
            curr_profP = (entry_price-exit_price)*100/entry_price
            if curr_pos == "Short": # exit short
                curr_profP *= -1
            entry_price = exit_price
            profP.append(curr_profP)
            profV.append(curr_profP*margin/100)
    elif i == len(cleaned_data)-1:
        curr_profP = (entry_price-exit_price)*100/entry_price
        if prev_pos == "Short":
            curr_profP *= -1
        profP.append(curr_profP)
        profV.append(curr_profP*margin/100)
    else:
        profP.append(None)
        profV.append(None)

cleaned_data['Profit Percentage'] = profP
cleaned_data['Profit Value'] = profV
iter = []
iterV = []


# Ensure 'Date' column is in datetime format
cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'])

# Find the minimum and maximum dates
min_date = cleaned_data['Date'].min()
max_date = cleaned_data['Date'].max()

# Calculate the total number of days between the min and max date
number_days = (max_date - min_date).days

# Extract year and month (year-month format)
cleaned_data['Year_Month'] = cleaned_data['Date'].dt.to_period('M')

# Find the number of unique year-month combinations
number_months = cleaned_data['Year_Month'].nunique()

cleaned_data = cleaned_data.drop(columns=['Year_Month'])
cleaned_data['Date'] = cleaned_data['Date'].astype(str)


# total profit percentage
profit_sum_p = cleaned_data['Profit Percentage'].sum()
iter.append('Profit Percentage')
iterV.append(profit_sum_p)

# total profit value
profit_sum_v = cleaned_data['Profit Value'].sum()
iter.append('Profit Value')
iterV.append(profit_sum_v)

# max profit percentage
profit_max_p = cleaned_data['Profit Percentage'].max()
iter.append('Max Profit Percentage')
iterV.append(profit_max_p)

# max loss percentage
profit_min_p = cleaned_data['Profit Percentage'].min()
iter.append('Min Profit Percentage')
iterV.append(profit_min_p)

# number of win days
num_win_days = (cleaned_data['Profit Percentage'] > 0).sum()
iter.append('Number of Win Days')
iterV.append(num_win_days)

# number of loss days
num_loss_days = (cleaned_data['Profit Percentage'] <= 0).sum()
iter.append('Number of Loss Days')
iterV.append(num_loss_days)

# average monthly profit value
avg_month_profit = profit_sum_v/number_months
iter.append('Average monthly profit value')
iterV.append(avg_month_profit)

# average monthly profit percent
avg_month_profit = profit_sum_p/number_months
iter.append('Average monthly profit percent')
iterV.append(avg_month_profit)

# average profit on win days
sum_win_profit_v = cleaned_data[cleaned_data['Profit Value'] >= 0]['Profit Value'].sum()
avg_win_profit_v = sum_win_profit_v/num_win_days
iter.append('Average win day profit value')
iterV.append(avg_win_profit_v)

# average profit on loss days
sum_win_profit_v = cleaned_data[cleaned_data['Profit Value'] < 0]['Profit Value'].sum()
avg_win_profit_v = sum_win_profit_v/num_win_days
iter.append('Average loss day profit value')
iterV.append(avg_win_profit_v)

# win days percent
win_days_percent = num_win_days/number_days*100
iter.append('Win day %')
iterV.append(win_days_percent)

# loss days percent
loss_days_percent = num_loss_days/number_days*100
iter.append('loss day %')
iterV.append(loss_days_percent)

# max win streak
max_win_streak = 0
current_streak = 0

# Loop through the values in the column
for value in cleaned_data['Profit Percentage']:
    if pd.notna(value) and value >= 0:
        current_streak += 1  # Increase streak for positive values
        max_win_streak = max(max_win_streak, current_streak)  # Update max streak if needed
    elif value < 0:
        current_streak = 0  # Reset streak for negative values
    # No action for NaN (empty) values, they won't break the streak
iter.append('Max W Streak')
iterV.append(max_win_streak)

# max loss streak
max_loss_streak = 0
current_streak = 0

# Loop through the values in the column
for value in cleaned_data['Profit Percentage']:
    if pd.notna(value) and value < 0:
        current_streak += 1  # Increase streak for positive values
        max_loss_streak = max(max_loss_streak, current_streak)  # Update max streak if needed
    elif value >= 0:
        current_streak = 0  # Reset streak for negative values
    # No action for NaN (empty) values, they won't break the streak
iter.append('Max L Streak')
iterV.append(max_loss_streak)

# max draw down
cleaned_data['curval'] = cleaned_data['Profit Value'] + margin
cleaned_data['curmax'] = cleaned_data['curval'].cummax()
cleaned_data['drawdown'] = (cleaned_data['curval'] - cleaned_data['curmax'])/cleaned_data['curmax']
max_drawdown = cleaned_data['drawdown'].min()*-100
iter.append('max drawdown')
iterV.append(max_drawdown)



""" FOR UPDATING GOOGLE SHEETS """
# Authenticate user
auth.authenticate_user()

# Get default credentials using google-auth
creds, project = default()

# Authorize gspread with the credentials
client = gspread.authorize(creds)

# Open the Google Sheet (replace "Your Google Sheet Name" with the actual sheet name)
sheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1m6kylRZBnOiUDqZe0-_viGpHWpDw08aHeT7mmAUw_Tg/edit?gid=1902718297#gid=1902718297").sheet1

# Handle NaN and infinity values by replacing them with None

data = cleaned_data.replace([float('inf'), float('-inf')], None)  # Replace infinities with None
data = data.fillna("")  # Replace NaNs with None
# data.index = data.index.astype(str)
# data = data.reset_index()



# Convert the DataFrame to a list of lists
data = data.values.tolist()

# Clear the existing data in the sheet
sheet.clear()

# Update the sheet with new data, including column headers
sheet.update('A1', [cleaned_data.columns.tolist()] + data)  # A1 will be the header

# Get all values of the specific column from Google Sheets
col_index = cleaned_data.columns.get_loc("Profit Percentage") + 1  # Adjust for 1-based index in Google Sheets
col_values = sheet.col_values(col_index)

# Find the next available row in the column
next_row = len(col_values) + 2

# Update the cell below the last row with the sum

for i in range(len(iter)):
    sheet.update_cell(next_row+i, col_index, iter[i])
    sheet.update_cell(next_row+i, col_index+1, str(iterV[i]))







<ipython-input-66-a09e14b1bec1>:114: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  cleaned_data['Year_Month'] = cleaned_data['Date'].dt.to_period('M')
<ipython-input-66-a09e14b1bec1>:254: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [cleaned_data.columns.tolist()] + data)  # A1 will be the header
